Project is inspired by [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/09/data-analysis-and-price-prediction-of-electric-vehicles/) and is an update on this [Kaggle dataset](https://www.kaggle.com/datasets/kkhandekar/cheapest-electric-cars)

In [12]:
import numpy as np
import regex as re
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
option = webdriver.ChromeOptions()
option.binary_location = chrome_path
option.add_argument('--disable-dev-shm-usage')
option.add_experimental_option("detach", True)
# option.add_argument('--load-extension='+u_block)

# Create new Instance of Chrome
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# enable browser logging
d = DesiredCapabilities.CHROME
d['goog:loggingPrefs'] = { 'browser':'ALL' }

In [13]:
# def seleniumCall():
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# browser = webdriver.Chrome(executable_path=driver_path, options=option, desired_capabilities=d);
wait = WebDriverWait(browser, timeout=10, poll_frequency=1)

url = 'https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~350|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all'
browser.get(url)
winds = browser.window_handles

browser.switch_to.window(winds[0])
browser.maximize_window()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:02<00:00, 3.49MB/s]


In [15]:
def clickXP(string):
    element = wait.until(EC.presence_of_element_located((By.XPATH, string)))
    return element    

def selectXP(string):
    # element = browser.find_element(By.XPATH,string)
    element = wait.until(EC.presence_of_element_located((By.XPATH, string)))
    return element

In [16]:
clickXP('//*[@id="evdb"]/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()

In [17]:
full_list = selectXP('//*[@id="evdb"]/main/div[2]/div[3]')

In [125]:
lst = full_list.find_elements(By.CLASS_NAME,'data-wrapper')

In [164]:
def findXPATH(element,xpath):
    return element.find_element(By.XPATH,xpath)

def findCLASS(element,class_name):
    return element.find_element(By.CLASS_NAME,class_name)

ev_list = []

for item in lst:
    ev_dict = {}
    title_wrap = findCLASS(item,'title-wrap')
    icons = findCLASS(item,'icons')
    specs = findCLASS(item,'specs')
    pricing = findCLASS(item,'pricing')

    title_split = title_wrap.text.split('\n')
    car = title_split[0]
    ev_dict['manufacturer'] = car.split(' ')[0]
    ev_dict['model'] = car.split(' ')[1]
    ev_dict['kwh'] = title_split[1]
    ev_dict['release'] = title_split[2]

    try: #not all cars have towing
        ev_dict['towweight'] = findCLASS(icons,'towweight').text
        ev_dict['drive'] = findXPATH(icons,'./span[4]').get_property('title')
        ev_dict['plug'] = findXPATH(icons,'./span[5]').get_attribute('textContent') #hidden element
        ev_dict['body'] = findXPATH(icons,'./span[6]').get_attribute('textContent')
        ev_dict['market_segment'] = findXPATH(icons,'./span[7]').text
        ev_dict['seats'] = findXPATH(icons,'./span[9]').text
    except: #if no towweight, span nums change
        ev_dict['towweight'] = 0
        ev_dict['drive'] = findXPATH(icons,'./span[2]').get_property('title')
        ev_dict['plug'] = findXPATH(icons,'./span[3]').get_attribute('textContent') #hidden element
        ev_dict['body'] = findXPATH(icons,'./span[4]').get_attribute('textContent')
        ev_dict['market_segment'] = findXPATH(icons,'./span[5]').text
        ev_dict['seats'] = findXPATH(icons,'./span[7]').text


    ev_dict['acceleration'] = findCLASS(specs,'acceleration').text
    ev_dict['max_speed'] = findCLASS(specs,'topspeed').text
    ev_dict['real_range'] =  findCLASS(specs,'erange_real').text
    ev_dict['efficiency'] =  findCLASS(specs,'efficiency').text
    ev_dict['fastcharge'] =  findXPATH(specs,'./p[5]/span[3]').get_attribute('textContent')

    pricing_split = pricing.text.split('\n')
    ev_dict['price_germany'] = pricing_split[0]
    ev_dict['price_ned'] = pricing_split[1]
    ev_dict['price_uk'] = pricing_split[2]
    
    ev_list.append(ev_dict)

AttributeError: 'NoneType' object has no attribute 'split'

In [154]:
ev_data = pd.DataFrame(data=ev_list,columns=list(ev_dict.keys()))

In [155]:
ev_data.head()

,manufacturer,model,kwh,release,towweight,drive,plug,body,market_segment,seats,acceleration,max_speed,real_range,efficiency,fastcharge,price_germany,price_ned,price_uk
0,Tesla,Model,75 kWh useable battery*,Available since February 2022,1600,All Wheel Drive,Type 2 CCS,SUV,D,5,5.0 sec,217 km/h,435 km,172 Wh/km,670,"€59,017","€54,993","£52,990"
1,Tesla,Model,57.5 kWh useable battery*,Available since November 2022,1600,Rear Wheel Drive,Type 2 CCS,SUV,D,5,6.9 sec,217 km/h,345 km,167 Wh/km,580,"€47,567","€47,993","£44,990"
2,Tesla,Model,57.5 kWh useable battery*,Available since December 2021,1000,Rear Wheel Drive,Type 2 CCS,Sedan,D,5,6.1 sec,225 km/h,380 km,151 Wh/km,630,"€46,667","€45,993","£42,990"
3,Tesla,Model,75 kWh useable battery*,Available since November 2021,1000,All Wheel Drive,Type 2 CCS,Sedan,D,5,4.4 sec,233 km/h,485 km,155 Wh/km,750,"€56,667","€53,993","£50,990"
4,BYD,ATTO,60 kWh useable battery*,Available since August 2022,NaN,Front Wheel Drive,Type 2 CCS,SUV,C,5,7.3 sec,160 km/h,320 km,188 Wh/km,300,"€44,625","€42,998",N/A


## Data Cleaning

In [165]:
ev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   manufacturer    298 non-null    object
 1   model           298 non-null    object
 2   kwh             298 non-null    object
 3   release         298 non-null    object
 4   towweight       194 non-null    object
 5   drive           298 non-null    object
 6   plug            298 non-null    object
 7   body            298 non-null    object
 8   market_segment  298 non-null    object
 9   seats           298 non-null    object
 10  acceleration    298 non-null    object
 11  max_speed       298 non-null    object
 12  real_range      298 non-null    object
 13  efficiency      298 non-null    object
 14  fastcharge      298 non-null    object
 15  price_germany   298 non-null    object
 16  price_ned       298 non-null    object
 17  price_uk        298 non-null    object
 18  towing    

To start, we want to transform 'release' column into:
 - 'released': Whether the car has been released or not
 - 'release_month': What month the car was/will be released
 - 'release_year': What year the car was/will be released

275 cars released, 23 yet to be released:

In [166]:
print('Released:', sum(ev_data['release'].str.contains('since')), '\nYet to be released:', sum(ev_data['release'].str.contains('from')))

Released: 275 
Yet to be released: 23


In [143]:
def getMonth(string):
    '''
    Description:
    For specific release string, retrieves the month that the car is released/to be released
    
    Parameters:
     - str - str type object
     
    Returns:
    str
    
    Example:
    >>> input:  yearDelay('Available since February 2022')
    >>> output: True
    '''
    return string[id2+2:-5]

In [ ]:
re.search('[\w*\s]{2}\w*','Available since February 2022').group()

In [ ]:
string_col = ['model']
categ_cols = ['manufacturer','released','release_month','release_year','drive','body','seats','market_segment','plug','towing']
num_cols = ['towweight','kwh','acceleration','max_speed','real_range','efficiency','fastcharge']
price_cols = ['price_germany','price_ned','price_uk']

In [ ]:
for price in price_cols:
    print(sum(ev_data[price]=='N/A'))

In [144]:
print(sum(ev_data.kwh.str.contains('useable battery*')),
sum(ev_data.acceleration.str.contains('sec')),
sum(ev_data.max_speed.str.contains('km.h')),
sum(ev_data.real_range.str.contains('km')),
sum(ev_data.efficiency.str.contains('Wh/km')),
sum(ev_data.fastcharge.str.contains('km/h')))

AttributeError: 'DataFrame' object has no attribute 'max_speed'

In [296]:
def numPart(string):
    if string == '-':
        return np.nan
    else:
        return re.search('[0-9,.]*',string).group()

In [357]:
def getPrice(string):
    if string == 'N/A':
        return np.nan
    else:
        string = string.replace('€','')
        string = string.replace('£','')
        string = string.replace(',','')
        string = string.replace('*','')
        string = string.strip()
        
        return string

In [350]:
string_cols = ['manufacturer','model','released','release_month','release_year']
num_cols = ['useable_battery','acceleration','max_speed','real_range',
             'efficiency','fastcharge']
price_cols = ['price_germany','price_ned','price_uk']

In [358]:
ev_data = pd.DataFrame(data=dic_list,columns=list(dic.keys()))
ev_data['manufacturer'] = ev_data['title'].apply(lambda x: x[:x.index(' ')])
ev_data['model'] = ev_data['title'].apply(lambda x: x[x.index(' ' ):])
ev_data['released'] = ev_data['date'].apply(lambda x: 1 if 'since' in x else 0)
ev_data['release_month'] = ev_data['date'].apply(lambda x: getMonth(x))
ev_data['release_year'] = ev_data['date'].apply(lambda x: x[-4:])

for col in num_cols:
    ev_data[col] = np.vectorize(numPart)(ev_data[col])
    
for price in price_cols:
    ev_data[price] = ev_data[price].apply(lambda x: getPrice(x))
    
for col in ['fastcharge']+price_cols:
    ev_data[col] = ev_data[col].apply(lambda x: np.nan if x=='nan' else x) 

col_order = string_cols+num_cols+price_cols

ev_data = ev_data[col_order]

In [359]:
for col in num_cols:
    try:
        ev_data[col] = ev_data[col].astype('Int64')
    except:
        ev_data[col] = ev_data[col].astype('float')

In [363]:
for price in price_cols:
    ev_data[price] = ev_data[price].astype('Int64')

In [377]:
ev_data[string_cols] = ev_data[string_cols].astype('category')

In [812]:
ev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   manufacturer     298 non-null    category
 1   model            298 non-null    object  
 2   released         298 non-null    category
 3   release_month    298 non-null    category
 4   release_year     298 non-null    category
 5   useable_battery  298 non-null    float64 
 6   acceleration     298 non-null    float64 
 7   max_speed        298 non-null    Int64   
 8   real_range       298 non-null    Int64   
 9   efficiency       298 non-null    Int64   
 10  fastcharge       295 non-null    Int64   
 11  price_germany    273 non-null    Int64   
 12  price_ned        266 non-null    Int64   
 13  price_uk         187 non-null    Int64   
dtypes: Int64(7), category(4), float64(2), object(1)
memory usage: 28.8+ KB


In [813]:
ev_data.to_csv('../data/ev_data.csv')